## Natural language inference

### Contradictory dear watson
***

Project: https://www.kaggle.com/competitions/contradictory-my-dear-watson

### Model: bert fine tunning

https://huggingface.co/distilbert/distilbert-base-multilingual-cased

__Objetivo__: testear el modelo entendiendo su funcionamiento, bias y limitaciones

This model is a distilled version of the BERT base multilingual model. The code for the distillation process can be found here. This model is cased: it does make a difference between english and English.

The model is trained on the concatenation of Wikipedia in 104 different languages listed here. The model has 6 layers, 768 dimension and 12 heads, totalizing 134M parameters (compared to 177M parameters for mBERT-base). On average, this model, referred to as DistilmBERT, is twice as fast as mBERT-base

**Importar Librerias**

In [2]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import seaborn as sns
import matplotlib.pyplot as plt
import os

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

TensorFlow version: 2.16.1
KerasNLP version: 0.14.4


__Conf__

In [3]:
try:
    # detect and init the TPU
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()  # default strategy if no TPU available

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1726181778.442075    1495 service.cc:145] XLA service 0x5b6d95e55a20 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726181778.442135    1495 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1726181778.442139    1495 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1726181778.442142    1495 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1726181778.442145    1495 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1726181778.442148    1495 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1726181778.442151    1495 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1726181778.442154    1495 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1726181778.442157    1495 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
RESULT_DICT = {
    0 : "entailment",
    1 : "neutral",
    2 : "contradiction"
}

#### Data

In [5]:
df_train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
df_test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [6]:
def display_pair_of_sentence(x):
    print( "Premise : " + x['premise'])
    print( "Hypothesis: " + x['hypothesis'])
    print( "Language: " + x['language'])
    print( "Label: " + str(x['label']))
    print()

df_train.head(5).apply(lambda x : display_pair_of_sentence(x), axis=1)

df_train.shape

Premise : and these comments were considered in formulating the interim rules.
Hypothesis: The rules developed in the interim were put together with these comments in mind.
Language: English
Label: 0

Premise : These are issues that we wrestle with in practice groups of law firms, she said. 
Hypothesis: Practice groups are not permitted to work on these issues.
Language: English
Label: 2

Premise : Des petites choses comme celles-là font une différence énorme dans ce que j'essaye de faire.
Hypothesis: J'essayais d'accomplir quelque chose.
Language: French
Label: 0

Premise : you know they can't really defend themselves like somebody grown uh say my age you know yeah
Hypothesis: They can't defend themselves because of their age.
Language: English
Label: 0

Premise : ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสดงออกและได้เล่นหลายบทบาทไปพร้อมกัน ๆ อาจช่วยให้เด็กจับความคล้ายคลึงและความแตกต่างระหว่างผู้คนในด้านความปรารถนา ความเชื่อ และความรู้สึกได้
Hypothesis: เด็กสามารถเห็นได้ว่าชาติพันธุ

(12120, 6)

In [7]:
df_train["premise_length"] = df_train["premise"].apply(lambda x : len(x))
df_train["hypothesis_length"] = df_train["hypothesis"].apply(lambda x : len(x))
df_train[["hypothesis_length", "premise_length"]].describe()

,hypothesis_length,premise_length
count,12120.000000,12120.000000
mean,53.892327,107.373185
std,25.302358,71.089954
min,4.000000,4.000000
25%,36.000000,55.000000
50%,51.000000,96.000000
75%,67.000000,146.000000
max,276.000000,967.000000


#### Data preprocessing

In [8]:
df_train = df_train[['premise', 'hypothesis', 'label']]
df_test = df_test[['premise', 'hypothesis']]

#### Train model

In [9]:
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer

In [10]:
model_name ='joeddav/xlm-roberta-large-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### Tokenize

In [11]:
def encode_premise_sentence(s):
    tokens=[]
    tokens.append('[CLS]')
    tokens+=list(tokenizer.tokenize(s))
    return tokenizer.convert_tokens_to_ids(tokens)

In [13]:
def encode_hypo_sentence(s):
    tokens=[]
    tokens.append('[sep]')
    tokens+=list(tokenizer.tokenize(s))
    tokens.append('[sep]')
    return tokenizer.convert_tokens_to_ids(tokens)

In [14]:
encode_hypo_sentence("jsalkgfad")

[3, 1647, 2317, 8517, 1021, 71, 3]

In [12]:
encode_premise_sentence("jsalkgfad")

[3, 1647, 2317, 8517, 1021, 71]

In [16]:
tokenized=[]
for i in range(len(df_train)):
    pre=encode_premise_sentence(df_train['premise'][i])
    hyp=encode_hypo_sentence(df_train['hypothesis'][i])
    tokenized.append(pre+hyp)
df_train['tokenized']=tokenized
df_train.head()

,premise,hypothesis,label,tokenized
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0,"[3, 136, 6097, 24626, 3542, 90698, 23, 26168, ..."
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2,"[3, 32255, 621, 37348, 450, 642, 148, 56644, 1..."
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0,"[3, 5581, 69332, 37899, 3739, 91362, 9, 16161,..."
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0,"[3, 398, 3714, 1836, 831, 25, 18, 6183, 65922,..."
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1,"[3, 6976, 114538, 171936, 18379, 101830, 14435..."


#### Attention Mask and Token Type ID

In [18]:
mask=[]
for i in range(len(df_train)):
    padded_seq=tokenizer(df_train['premise'][i],df_train['hypothesis'][i], padding=True,add_special_tokens = True)
    mask.append(padded_seq)
df_train['masked'] = mask
df_train.head(5)
# print(mask[0])

,premise,hypothesis,label,tokenized,masked
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0,"[3, 136, 6097, 24626, 3542, 90698, 23, 26168, ...","[input_ids, attention_mask]"
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2,"[3, 32255, 621, 37348, 450, 642, 148, 56644, 1...","[input_ids, attention_mask]"
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0,"[3, 5581, 69332, 37899, 3739, 91362, 9, 16161,...","[input_ids, attention_mask]"
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0,"[3, 398, 3714, 1836, 831, 25, 18, 6183, 65922,...","[input_ids, attention_mask]"
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1,"[3, 6976, 114538, 171936, 18379, 101830, 14435...","[input_ids, attention_mask]"


#### Train Model

In [19]:
max_len=237
def build_model():
    bert_encoder = TFAutoModel.from_pretrained('joeddav/xlm-roberta-large-xnli')
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    embedding = bert_encoder([input_word_ids, input_mask])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [20]:
def input_convert(data):
    inputs={
        'input_word_ids':[],
        'input_mask':[]
    }
    for each in data:
        inputs['input_word_ids'].append(each['input_ids'])
        inputs['input_mask'].append(each['attention_mask'])
        
    inputs['input_word_ids']= tf.ragged.constant( inputs['input_word_ids']).to_tensor()
    inputs['input_mask']= tf.ragged.constant( inputs['input_mask']).to_tensor()
    return inputs

In [21]:
train_input=input_convert(df_train['masked'].values)
for key in train_input.keys():
    train_input[key] = train_input[key][:,:max_len]

In [22]:
train_input

{'input_word_ids': <tf.Tensor: shape=(12120, 237), dtype=int32, numpy=
 array([[    0,   136,  6097, ...,     0,     0,     0],
        [    0, 32255,   621, ...,     0,     0,     0],
        [    0,  5581, 69332, ...,     0,     0,     0],
        ...,
        [    0,   581,  5526, ...,     0,     0,     0],
        [    0,  1913,    70, ...,     0,     0,     0],
        [    0,  1326, 66570, ...,     0,     0,     0]], dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(12120, 237), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [26]:
from transformers import TFXLMRobertaModel

In [29]:
import tensorflow as tf
from transformers import TFXLMRobertaModel

def build_model():
    max_len = 100  # Adjust as needed

    # Define input layers using tf.keras.Input
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")

    # Instantiate the transformer model
    bert_encoder = TFXLMRobertaModel.from_pretrained('joeddav/xlm-roberta-large-xnli')

    # Forward pass through the model
    # Ensure inputs are handled correctly
    encoder_outputs = bert_encoder(
        input_ids=input_word_ids,
        attention_mask=input_mask
    )
    embedding = encoder_outputs.last_hidden_state  # Use last_hidden_state for embeddings

    # Define the output layer
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:, 0, :])

    # Build and compile the model
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Adjust learning rate as needed
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [31]:
import tensorflow as tf
from transformers import TFXLMRobertaModel

def build_model():
    max_len = 100  # Adjust as needed

    # Define input layers using tf.keras.Input
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")

    # Instantiate the transformer model
    bert_encoder = TFXLMRobertaModel.from_pretrained('joeddav/xlm-roberta-large-xnli')

    # Use tf.function to convert Keras tensors to TensorFlow tensors
    @tf.function
    def get_model_outputs(input_ids, attention_mask):
        return bert_encoder(input_ids=input_ids, attention_mask=attention_mask)

    # Forward pass through the model
    encoder_outputs = get_model_outputs(input_word_ids, input_mask)
    embedding = encoder_outputs.last_hidden_state  # Use last_hidden_state for embeddings

    # Define the output layer
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:, 0, :])

    # Build and compile the model
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Adjust learning rate as needed
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [33]:
 import tensorflow as tf
from transformers import TFXLMRobertaModel

def build_model():
    max_len = 100  # Adjust this as needed

    # Define input layers using tf.keras.Input
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")

    # Instantiate the transformer model
    bert_encoder = TFXLMRobertaModel.from_pretrained('joeddav/xlm-roberta-large-xnli')

    # Define a function to call the model
    def call_transformer_model(input_ids, attention_mask):
        # Ensure the inputs are TensorFlow tensors
        return bert_encoder(input_ids=input_ids, attention_mask=attention_mask)
    
    # Call the model
    encoder_outputs = call_transformer_model(input_word_ids, input_mask)
    embedding = encoder_outputs.last_hidden_state  # Get the last hidden state

    # Define the output layer
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:, 0, :])  # Using the first token (CLS token) for classification

    # Build and compile the model
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Adjust learning rate as needed
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [35]:
# Example of preparing data
train_input = {
    'input_word_ids': tf.convert_to_tensor(train_enc['input_ids'], dtype=tf.int32),
    'input_mask': tf.convert_to_tensor(train_enc['attention_mask'], dtype=tf.int32)
}

# Example of model training
model = build_model()
model.summary()
model.fit(train_input, df_train['label'].values, epochs=5, verbose=1, batch_size=128, validation_split=0.1, callbacks=[early_stop])


NameError: name 'train_enc' is not defined

In [34]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
with strategy.scope():
    model = build_model()
    model.summary()
    model.fit(train_input, df_train['label'].values, epochs=5, verbose=1, batch_size=128, validation_split=0.1, callbacks=[early_stop])

Some layers from the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing TFXLMRobertaModel: ['classifier']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at joeddav/xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


ValueError: Exception encountered when calling layer 'tfxlm_roberta_model_5' (type TFXLMRobertaModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tfxlm_roberta_model_5' (type TFXLMRobertaModel):
  • input_ids=<KerasTensor shape=(None, 100), dtype=int32, sparse=False, name=input_word_ids>
  • attention_mask=<KerasTensor shape=(None, 100), dtype=int32, sparse=False, name=input_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=Fal

#### Prediction

In [ ]:
mask=[]
for i in range(len(df_test)):
    padded_seq=tokenizer(df_test['premise'][i],df_test['hypothesis'][i],
                        padding=True,add_special_tokens =True)
    mask.append(padded_seq)
df_test['masked']=mask
df_test.head()

In [ ]:
predictions=[np.argmax(i) for i in model.predict(test_input)]

In [ ]:
VALIDATION_SPLIT = 0.2
TRAIN_SIZE = int(df_train.shape[0]*(1-VALIDATION_SPLIT))
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [ ]:
def split_labels(x, y):
    return (x[0], x[1]), y


training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[['premise','hypothesis']].values,
            keras.utils.to_categorical(df_train['label'], num_classes=3)
        )
    )
)

train_dataset = training_dataset.take(TRAIN_SIZE)
val_dataset = training_dataset.skip(TRAIN_SIZE)

# Apply the preprocessor to every sample of train, val and test data using `map()`.
# [`tf.data.AUTOTUNE`](https://www.tensorflow.org/api_docs/python/tf/data/AUTOTUNE) and `prefetch()` are options to tune performance, see
# https://www.tensorflow.org/guide/data_performance for details.

train_preprocessed = train_dataset.map(split_labels, tf.data.AUTOTUNE).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
val_preprocessed = val_dataset.map(split_labels, tf.data.AUTOTUNE).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
# Load a BERT model.
with strategy.scope():
    classifier = keras_nlp.models.DistilBertClassifier.from_preset("distil_bert_base_multi", num_classes=3)

    # in distributed training, the recommendation is to scale batch size and learning rate with the numer of workers.
    classifier.compile(optimizer=keras.optimizers.Adam(1e-5*strategy.num_replicas_in_sync),
                       loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    
    classifier.summary()

__Fine tunning distil bert__

In [ ]:
EPOCHS=10
history = classifier.fit(train_preprocessed,
                         epochs=EPOCHS,
                         validation_data=val_preprocessed
                        )